# ETL Básico - New York (Google Maps)

*Nota:* Para realizar este proceso es importante haber ejecutado el archivo `FILTER-metadata-sitios.ipynb` en este mismo directorio, ya que se necesitan TODOS los restaurantes de EEUU.

En este archivo se enfoca en realizar la unificación de todos los archivos en el directorio `data/raw/reviews-estados/review-New_York` y filtrar sólo las reviews de restaurantes en New York con el objetivo de exportarlas junto con otro dataset que corresponde a los restaurantes de dicho estado para luego hacer un EDA y un ETL profundos.

In [1]:
import pandas as pd
import numpy as np
import warnings
import sys
import os

# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [2]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '../..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

# importar librerías personalizadas de ./src/utils/EDA.py
from src.utils.EDA import merge_files

## Se importan los DataSets necesarios
Se importan los datos de todos los locales en Nueva York y de todos los restaurantes de EEUU.

In [3]:
# se define la ruta absoluta de los archivos
file_path = os.path.join(project_root, 'data/raw/google_maps/reviews-estados/review-New_York')

# se almacenan todos los registros en un df con la ayuda de la función merge_files(path)
new_york = merge_files(file_path)

Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/reviews-estados/review-New_York\1.parquet
Archivo 1.parquet leído exitosamente con 150000 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/reviews-estados/review-New_York\10.parquet
Archivo 10.parquet leído exitosamente con 150000 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/reviews-estados/review-New_York\11.parquet
Archivo 11.parquet leído exitosamente con 150000 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/reviews-estados/review-New_York\12.parquet
Archivo 12.parquet leído exitosamente con 150000 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/reviews-estados/review-New_York\13.parquet
Archivo 13.parquet leído exitosamente con 150000 registros.
Leyendo archivo: d:\Henry-D

In [4]:
# se visualizan las primeras tres filas del dataframe
new_york.head(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.137221e+20,Alvin Martinez,1603494795361,5,I'm late to posting this but this store especi...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c25fc9494dce47:0x6d63c807b59a55
1,1.072934e+20,Johnnie Jackson,1620157037403,1,Very dissatisfied I did not get my phone the p...,None,{'text': 'We pride ourselves on providing an a...,0x89c25fc9494dce47:0x6d63c807b59a55
2,1.003786e+20,Manie Blazer,1597431662039,5,Excellent very well done with professional car...,None,None,0x89c25fc9494dce47:0x6d63c807b59a55


In [5]:
# se define la ruta absoluta del archivo con los restaurantes de todo el país
file_path = os.path.join(project_root, 'data/raw/google_maps/restaurantes-EEUU.parquet')

# se lee el datasets de restaurantes en EEUU
restaurantes_EEUU = pd.read_parquet(file_path)

# se visualizan primeras tres filas
restaurantes_EEUU.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
2,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...


## Se filtran los dataframes
De los restaurantes de EEUU se filtran los restaurantes de Nueva York y, de todos los locales de Nueva York, se filtran sólo los restaurantes en dos dataframes separados. El punto en común entre ambos es la columna `gmap_id`.

In [9]:
# se filtran los restaurantes de Nueva York
restaurantes_ny = restaurantes_EEUU[restaurantes_EEUU['gmap_id'].isin(new_york['gmap_id'])]

# contar cantidad de registros y visualizar primeras filas
print(len(restaurantes_ny))
restaurantes_ny.head(3)

9711


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
20,Dunkin',"Dunkin', 4008 Bell Blvd, Queens, NY 11361",0x89c261f60bdf13db:0x38da730e4687a97b,Long-running chain serving signature breakfast...,40.763985,-73.771430,"[Coffee shop, Bagel shop, Bakery, Breakfast re...",3.5,8,$,"[[Thursday, 6AM–7PM], [Friday, 6AM–7PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 7PM,"[0x89c3ab9229879ec3:0x3f4b2b46d7d2c503, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
26,Raffaello Kosher Pizza,"Raffaello Kosher Pizza, 37 W 46th St, New York...",0x89c258ffaeaba947:0x8355860772a595a9,This casual joint serves up traditional Italia...,40.756872,-73.980427,[Restaurant],3.4,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c2f41bdce4dc0f:0xf4e3f717a4950ea3, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
39,Kennedy's Chicken & Sandwiches,"Kennedy's Chicken & Sandwiches, 495 E 138th St...",0x89c2f5c3caef3fb5:0xb7f855503b4bc974,None,40.808010,-73.919617,[Fast food restaurant],3.7,8,None,"[[Thursday, 8AM–3AM], [Friday, 8AM–3AM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Closed ⋅ Opens 8AM,"[0x89c2f5cfbcdeb7c1:0x1c3c27e3a6bcdab6, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...


In [10]:
# Filtrar las reviews de los restaurantes en Nueva York
reviews_restaurantes_ny = new_york[new_york['gmap_id'].isin(restaurantes_ny['gmap_id'])]

#contar cantidad de registros y visualizar primeras filas
print(len(reviews_restaurantes_ny))
reviews_restaurantes_ny.head(3)

675027


,user_id,name,time,rating,text,pics,resp,gmap_id
147,1.018635e+20,Maria Patricia Londoño,1629077998394,4,The donuts is always a good place to buy somet...,None,None,0x89c261f60bdf13db:0x38da730e4687a97b
148,1.019546e+20,Kristal,1593616678555,5,I went into this Dunkin' yesterday and got som...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c261f60bdf13db:0x38da730e4687a97b
149,1.089877e+20,Efrain Hernandez,1594150943631,3,Bought a Machiato and as soon as I touched the...,None,None,0x89c261f60bdf13db:0x38da730e4687a97b


De los 161696 restaurantes en EEUU que aún siguen abiertos al público (o al menos no tienen el valor "Permanently closed" en la columna "state"), sólo 9711 son de Nueva York, y estos tienen un total de 675027 reviews

## Se exportan los datasets de Restaurantes en Nueva York y las Reviews en estos por separado

In [13]:
# se determina la ruta para exportar los datos de los restaurantes y las reviews
restaurantes_ny_path = os.path.join(project_root, 'data/raw/google_maps/restaurantes-NY.parquet')
reviews_restaurantes_ny_path = os.path.join(project_root, 'data/raw/google_maps/reviews-estados/reviews-restaurantes-NY.parquet')

# se exporta el df con restaurantes de Nueva York a un archivo parquet
restaurantes_ny.to_parquet(restaurantes_ny_path, index=False, compression='gzip')
# se exporta el df con las reviews de los restaurantes de Nueva York a un archivo parquet
restaurantes_ny.to_parquet(reviews_restaurantes_ny_path, index=False, compression='gzip')